In [48]:
import numpy as np 
import pandas as pd 


In [49]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [50]:
movies.head(1)

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                      homepage     id  \
0  http://www.avatarmovie.com/  19995   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   

  original_title                                           overview  \
0         Avatar  In the 22nd century, a paraplegic Marine is di...   

   popularity                               production_companies  \
0  150.437577  [{"name": "Ingenious Film Partners", "id": 289...   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   

                       tagline   title  vote_average  vote_count  
0  Enter the World of Pandora.  Avatar           7.2       11800

In [51]:
movies.shape

(4803, 20)

In [52]:
movies = movies.merge(credits, on='title')

movies.shape

(4809, 23)

In [53]:


movies=movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [54]:
import ast

In [55]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [56]:

movies.shape

(4809, 7)

In [57]:
import ast

def convert3(lst):
    L = []
    counter = 0
    for i in lst:
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(lst):
    L = []
    for i in lst:
        if isinstance(i, dict) and i.get('job') == 'Director':
            L.append(i['name'])
    return L

# Assuming your columns are already lists of dicts
movies['genres'] = movies['genres'].apply(lambda x: [i['name'] for i in x] if x and isinstance(x[0], dict) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: [i['name'] for i in x] if x and isinstance(x[0], dict) else x)
movies['cast'] = movies['cast'].apply(lambda x: convert3(x) if x and isinstance(x[0], dict) else x)
movies['crew'] = movies['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
movies['crew'] = movies['crew'].apply(fetch_director)


In [58]:
movies.head(1)
movies.shape

(4809, 7)

In [59]:


# Convert stringified lists into actual Python lists
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
movies['keywords'] = movies['keywords'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
movies['cast'] = movies['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
movies['crew'] = movies['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Now remove spaces, only for string items
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") if isinstance(i, str) else i for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") if isinstance(i, str) else i for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") if isinstance(i, str) else i for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") if isinstance(i, str) else i for i in x])


In [60]:
movies.head(1)

movie_id   title                                           overview  \
0     19995  Avatar  In the 22nd century, a paraplegic Marine is di...   

                                              genres  \
0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   

                                            keywords  \
0  [{'id': 1463, 'name': 'culture clash'}, {'id':...   

                                                cast            crew  
0  [{'cast_id': 242, 'character': 'Jake Sully', '...  [JamesCameron]

In [61]:
movies['tags'] = movies.apply(lambda row: [row['overview']] + row['genres'] + row['keywords'] + row['cast'] + row['crew'], axis=1)

In [62]:
movies.head(1)

movie_id   title                                           overview  \
0     19995  Avatar  In the 22nd century, a paraplegic Marine is di...   

                                              genres  \
0  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...   

                                            keywords  \
0  [{'id': 1463, 'name': 'culture clash'}, {'id':...   

                                                cast            crew  \
0  [{'cast_id': 242, 'character': 'Jake Sully', '...  [JamesCameron]   

                                                tags  
0  [In the 22nd century, a paraplegic Marine is d...

In [63]:
new_df = movies[['movie_id', 'title', 'tags']]

In [64]:
new_df

movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                                   tags  
0     [In the 22nd century, a paraplegic Marine is d...  
1     [Captain Barbossa, long believed to be dead, h...  
2     [A cryptic message from Bond’s past sends him ...  
3     [Following the death of District Attorney Harv...  
4     [John Carter is a war-weary, former military c...  
...                                                 ...  
4804  [El Mariachi just wants to play his guitar and...  
4805  [A newlywed couple's honeymoon is upended by t...  
4806  ["Signed, Sealed, Delivered" introduces a dedi...  
4807  [When ambitious New York attorney Sam is sent ...  
4808  [Ever since the second grade when he first saw...  

[4809 rows x 3 columns]

In [65]:
new_df['tags'] = new_df['tags'].apply(
    lambda x: " ".join(map(str, x)) if isinstance(x, list) else str(x)
)

C:\Users\khanr\AppData\Local\Temp\ipykernel_1548\331670952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(


In [66]:
new_df.head(1)

movie_id   title                                               tags
0     19995  Avatar  In the 22nd century, a paraplegic Marine is di...

In [67]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. {\'id\': 28, \'name\': \'Action\'} {\'id\': 12, \'name\': \'Adventure\'} {\'id\': 14, \'name\': \'Fantasy\'} {\'id\': 878, \'name\': \'Science Fiction\'} {\'id\': 1463, \'name\': \'culture clash\'} {\'id\': 2964, \'name\': \'future\'} {\'id\': 3386, \'name\': \'space war\'} {\'id\': 3388, \'name\': \'space colony\'} {\'id\': 3679, \'name\': \'society\'} {\'id\': 3801, \'name\': \'space travel\'} {\'id\': 9685, \'name\': \'futuristic\'} {\'id\': 9840, \'name\': \'romance\'} {\'id\': 9882, \'name\': \'space\'} {\'id\': 9951, \'name\': \'alien\'} {\'id\': 10148, \'name\': \'tribe\'} {\'id\': 10158, \'name\': \'alien planet\'} {\'id\': 10987, \'name\': \'cgi\'} {\'id\': 11399, \'name\': \'marine\'} {\'id\': 13065, \'name\': \'soldier\'} {\'id\': 14643, \'name\': \'battle\'} {\'id\': 14720, \'name\': \'love affair\'}

In [70]:
%pip install nltk
import nltk
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer() 

  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 254.4 kB/s eta 0:00:04
   ------------- -------------------------- 0.5/1.5 MB 254.4 kB/s eta 0:00:04
   ------------- --------------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [72]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
    
      

In [73]:
new_df['tags'] = new_df['tags'].apply(stem)

C:\Users\khanr\AppData\Local\Temp\ipykernel_1548\3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [75]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())  

C:\Users\khanr\AppData\Local\Temp\ipykernel_1548\3344500878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [76]:
new_df.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  in the 22nd century, a parapleg marin is dispa...  
1  captain barbossa, long believ to be dead, ha c...  
2  a cryptic messag from bond’ past send him on a...  
3  follow the death of district attorney harvey d...  
4  john carter is a war-weary, former militari ca...

In [77]:
new_df['tags'][1]

'captain barbossa, long believ to be dead, ha come back to life and is head to the edg of the earth with will turner and elizabeth swann. but noth is quit as it seems. {\'id\': 12, \'name\': \'adventure\'} {\'id\': 14, \'name\': \'fantasy\'} {\'id\': 28, \'name\': \'action\'} {\'id\': 270, \'name\': \'ocean\'} {\'id\': 726, \'name\': \'drug abuse\'} {\'id\': 911, \'name\': \'exot island\'} {\'id\': 1319, \'name\': \'east india trade company\'} {\'id\': 2038, \'name\': "love of one\' life"} {\'id\': 2052, \'name\': \'traitor\'} {\'id\': 2580, \'name\': \'shipwreck\'} {\'id\': 2660, \'name\': \'strong woman\'} {\'id\': 3799, \'name\': \'ship\'} {\'id\': 5740, \'name\': \'alliance\'} {\'id\': 5941, \'name\': \'calypso\'} {\'id\': 6155, \'name\': \'afterlife\'} {\'id\': 6211, \'name\': \'fighter\'} {\'id\': 12988, \'name\': \'pirate\'} {\'id\': 157186, \'name\': \'swashbuckler\'} {\'id\': 179430, \'name\': \'aftercreditsstinger\'} {\'cast_id\': 4, \'character\': \'captain jack sparrow\', \

In [78]:
%pip install scikit-learn

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [80]:
vectors 

array([[0, 2, 1, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], shape=(4809, 5000))

In [81]:
vectors[0]

array([0, 2, 1, ..., 0, 0, 0], shape=(5000,))

In [82]:
cv.get_feature_names_out()

array(['000', '10', '100', ..., 'zoo', 'zooey', 'zoë'],
      shape=(5000,), dtype=object)

In [85]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity[0]
sorted(similarity[0])

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0004494106517960526),
 np.float64(0.0006547296237498641),
 np.float64(0.001944447778548432),
 np.float64(0.00324074629758072),
 np.float64(0.0032926015309451893),
 np.float64(0.0041837854799378245),
 np.float64(0.004205747759342412),
 np.float64(0.007752615869443367),
 np.float64(0.051988129948031994),
 np.float64(0.05244416617564631),
 np.float64(0.05509268705887224),
 np.float64(0.061136174532500906),
 np.float64(0.0642364891516198),
 np.float64(0.07365591237289693),
 np.float64(0.07583623318143544),
 np.float64(0.08325978382019104),
 np.float64(0.08488761431773555),
 np.float64(0.08514077769870654),
 np.float64(0.0872456036043393),
 np.float64(0.10037005132085416),
 np.float64(0.11201299426104513),
 np.float64(0.12264287133489578),
 np.float64(0.12411113418659078),
 np.float64(0.1385526552681904),
 np.float64(0.15062261621327838),
 np.float64(0.1665195676403821),

In [88]:
sorted(list(enumerate(similarity[0])),reverse = True, key=lambda x: x[1])[1:6]

[(287, np.float64(0.9888000224605421)),
 (281, np.float64(0.9885425231296665)),
 (65, np.float64(0.9884930521853564)),
 (849, np.float64(0.9883878585784034)),
 (1407, np.float64(0.9883258731213522))]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
                         
    
   

In [92]:
recommend('Batman Begins')

The Dark Knight
Batman v Superman: Dawn of Justice
The Pianist
The Fifth Element
Gandhi


In [93]:
new_df.iloc[1216].title

'Autumn in New York'

In [ ]:
import pickle
pickle.dump(new_df, open('movies.pkl','wb'))


In [95]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], shape=(4809,), dtype=object)

In [99]:
pickle.dump(new_df.to_dict(), open('movie_dict.pkl','wb'))

In [100]:
pickle.dump(similarity, open('similarity.pkl','wb'))